In [1]:

# coding: utf-8

# In[ ]:


from __future__ import print_function
import asyncio

import sys, os
import numpy as np
import struct
from time import time
import matplotlib.pyplot as plt 

from middleware_v0 import manage_tasks, req_kernel
from ip import ip0, ip1, ip2, ip3, Buffer0_A, Buffer0_B, Buffer1_A, Buffer1_B, Buffer2_A, Buffer2_B, Buffer3_A, Buffer3_B

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'


async def main():
    # Start kernel manager
    task_manager = asyncio.create_task(manage_tasks())

    ntt_in  = [0 for i in range(1024)]
    intt_in = [0 for i in range(1024)]
    fft_in  = [0 for i in range(1024)]
    ifft_in = [0 for i in range(1024)]
    tmp     = [0 for i in range(1024)]

    # =============== NTT test vector =================
    with open("NTT_in.txt", "r") as NTT_in:
        for i in range(1024):
            ntt_in[i] = np.uint16(NTT_in.readline())
    with open("NTT_out.txt", "r") as iNTT_in:
        for i in range(1024):
            intt_in[i] = np.uint16(iNTT_in.readline())
    # =============== FFT test vector =================
    with open("FFT_in.txt", "r") as FFT_in:
        for i in range(1024):
            fft_in[i] = float(FFT_in.readline())
    with open("FFT_out.txt", "r") as iFFT_in:
        for i in range(1024):
            ifft_in[i] = float(iFFT_in.readline())

    # Start Falcon-related tasks
    task1 = req_kernel(ntt_in, tmp, 0x2)
    task2 = req_kernel(intt_in, tmp, 0x3)
    task3 = req_kernel(fft_in, tmp, 0x0)
    task4 = req_kernel(ifft_in, tmp, 0x1)
    task5 = req_kernel(ntt_in, tmp, 0x2) # ntt_out
    task6 = req_kernel(intt_in, tmp, 0x3) #intt_out
    task7 = req_kernel(fft_in, tmp, 0x0) # fft_out
    task8 = req_kernel(ifft_in, tmp, 0x1) # ifft_out

    
    # ================== Compare ======================
    print("-------------------------------")
    print("Compare NTT with golden vector:")
    error_cnt = 0
    
    ntt_out = await task5
    timeKernelStart = time()
    
    with open("NTT_out.txt", "r") as NTT_out:
        for i in range(1024):
            golden = np.uint16(NTT_out.readline())
            if ntt_out[i] != golden:
                print(f"{i:<4} Failed!! Expect value: {golden}. Actual value: {ntt_out[i]}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    print("-------------------------------")
    print("Compare iNTT with golden vector:")
    error_cnt = 0
    
    intt_out = await task6
    with open("NTT_in.txt", "r") as iNTT_out:
        for i in range(1024):
            golden = np.uint16(iNTT_out.readline())
            if intt_out[i] != golden:
                print(f"{i:<4} Failed!! Expect value: {golden}. Actual value: {intt_out[i]}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    print("-------------------------------")
    print("Compare FFT with golden vector:")
    error_cnt = 0
    round_factor = 6
    
    fft_out = await task7
    with open("FFT_out.txt", "r") as FFT_out:
        for i in range(1024):
            golden = float(FFT_out.readline())
            if round(fft_out[i], round_factor) != round(golden, round_factor):
                print(f"{i:<4}Failed!! Expect value: {golden:.17f}. Actual value: {fft_out[i]:.17f}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    print("-------------------------------")
    print("Compare iFFT with golden vector:")
    error_cnt = 0
    round_factor = 6
    
    ifft_out = await task8
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    with open("FFT_in.txt", "r") as iFFT_out:
        for i in range(1024):
            golden = float(iFFT_out.readline())
            if round(ifft_out[i], round_factor) != round(golden, round_factor):
                print(f"{i:<4}Failed!! Expect value: {golden:.17f}. Actual value: {ifft_out[i]:.17f}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")

    # Close FFT manager when done
    task_manager.cancel()


if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    print("===============================")
    asyncio.run(main())
    print("===============================")
    print("Exit process")
    


Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
Kernel execution time: 0.8766565322875977 s
-------------------------------
Compare NTT with golden vector:
Test Passed!!
-------------------------------
Compare iNTT with golden vector:
Test Passed!!
-------------------------------
Compare FFT with golden vector:
Test Passed!!
-------------------------------
Compare iFFT with golden vector:
Test Passed!!
Exit process
